In [1]:
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
# Caminho para os logs do tensorboard e nome do modelo
path, name = "logs", "Model2"

# Importar os dados

In [3]:
test_data = np.load("Test_Data.npy", allow_pickle=True)
train_data = np.load("Train_Data.npy", allow_pickle=True)

Normalizar os dados entre 0 e 1 utilizando máximo e mínimo?
Normalização linear com picos nos dados poderá diluir o resto dos dados?

In [42]:
# TODO: normalizar os dados
x_train, y_train, x_test, y_test = [], [], [], []

for feature, label in train_data:
    x_train.append(feature)
    y_train.append(label)
    
for feature, label in test_data:
    x_test.append(feature)
    y_test.append(label)
    
x_train = np.array(x_train).reshape(-1, 69)
y_train = np.array(y_train)
x_test = np.array(x_test).reshape(-1, 69)
y_test = np.array(y_test)

In [43]:
print(f"Train features: {x_train.shape} | Train_labels: {y_train.shape}")
print(f"Test features: {x_test.shape} | Test_labels: {y_test.shape}")

Train features: (1345389, 69) | Train_labels: (1345389,)
Test features: (149487, 69) | Test_labels: (149487,)


# Criar o modelo

Cada elemento tem 69 parâmetros que o modelo pode avaliar, logo terá um input layer com 69 entradas

In [50]:
inputs = keras.Input(shape=(69,))
fc1 = keras.layers.Dense(100, activation='relu')(inputs)
d1 = keras.layers.Dropout(0.1)(fc1, training=True)
# bn1 = keras.layers.BatchNormalization()(d1)
fc2 = keras.layers.Dense(150, activation='relu')(d1)
d2 = keras.layers.Dropout(0.1)(fc2, training=True)
# bn2 = keras.layers.BatchNormalization()(d2)
fc3 = keras.layers.Dense(200, activation='relu')(d2)
d3 = keras.layers.Dropout(0.1)(fc3, training=True)
# bn3 = keras.layers.BatchNormalization()(d3)
fc4 = keras.layers.Dense(120, activation='relu')(d3)
d4 = keras.layers.Dropout(0.1)(fc4, training=True)
# bn4 = keras.layers.BatchNormalization()(d4)
fc5 = keras.layers.Dense(60, activation='relu')(d4)
# bn5 = keras.layers.BatchNormalization()(fc5)
outputs = keras.layers.Dense(2, activation='softmax')(d5)
model = keras.Model(inputs, outputs)

model.summary()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 69)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 100)               7000      
_________________________________________________________________
dropout_21 (Dropout)         (None, 100)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 100)               400       
_________________________________________________________________
dense_32 (Dense)             (None, 150)               15150     
_________________________________________________________________
dropout_22 (Dropout)         (None, 150)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 150)               600 

# Callbacks

In [52]:
# Tensorboard
TB = keras.callbacks.TensorBoard(path+"/"+name)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, verbose=2, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("Models/"+name+".h5", save_best_only=True, 
                                     monitor="val_loss", mode="min")

# Treinar modelo

In [53]:
model.fit(x_train, y_train, validation_split=0.1, shuffle=True,
          batch_size=64, epochs=20, verbose=2, callbacks=[TB, ES, MC])

Train on 1210850 samples, validate on 134539 samples
Epoch 1/20
 - 130s - loss: 0.0012 - accuracy: 0.9997 - val_loss: 4.6932e-04 - val_accuracy: 1.0000
Epoch 2/20
 - 130s - loss: 3.0039e-04 - accuracy: 0.9999 - val_loss: 0.0013 - val_accuracy: 0.9997
Epoch 3/20
 - 132s - loss: 2.9344e-04 - accuracy: 0.9999 - val_loss: 0.0057 - val_accuracy: 0.9999
Epoch 4/20
 - 129s - loss: 2.3489e-04 - accuracy: 0.9999 - val_loss: 0.0499 - val_accuracy: 0.9999
Epoch 5/20
 - 130s - loss: 2.1300e-04 - accuracy: 1.0000 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 6/20
 - 129s - loss: 2.2490e-04 - accuracy: 1.0000 - val_loss: 0.0369 - val_accuracy: 0.9999
Epoch 7/20
 - 128s - loss: 1.7706e-04 - accuracy: 1.0000 - val_loss: 0.0398 - val_accuracy: 1.0000
Epoch 00007: early stopping


In [57]:
model.save("Models/" + name + ".h5")